In [ ]:
# Embedding

In [16]:
import os
import pandas as pd
import sys
import re

from dotenv import load_dotenv
from openai import AzureOpenAI

from langchain_openai import AzureChatOpenAI

import csv
import time
from datetime import date, datetime
print('Python version:', sys.version)
print('\nWorking directory:', os.getcwd())
os.chdir(r'C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Topics')
print('\nCurrent working directory:', os.getcwd())

print("Last run:", date.today())

Python version: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]

Working directory: C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Topics

Current working directory: C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Topics
Last run: 2025-11-09


# Get and explore topics definition

In [19]:
file_path = 'C:\\Users\\Denis_Davydov2\\OneDrive - EPAM\\Prophet_AI_docs\\Datasets\\AI_skills\\Topics\\'
file_name = 'topics_definitions_all.csv'


topics_all = pd.read_csv(file_path+file_name)
print("File read: ", file_name)
topics_all

File read:  topics_definitions_all.csv


,id,term,definition
0,1,.net,"In the context of AI, .NET is a software devel..."
1,2,/invocations endpoint,"In the context of AI, the ""/invocations endpoi..."
2,3,/ping endpoint,"In the context of AI, a /ping endpoint is a sp..."
3,6,1-bit inference,"In the context of AI, 1-bit inference refers t..."
4,7,1-bit llm,A 1-bit large language model (LLM) is a highly...
...,...,...,...
19009,19018,zmp control,ZMP control (Zero Moment Point control) in AI/...
19010,19019,zombie startups,"In AI/ML, ""zombie startups"" are companies that..."
19011,19020,zone-based navigation,Zone-based navigation is an AI/ML approach tha...
19012,19021,zoom,"In AI/ML, ""zoom"" denotes a geometric image tra..."


## Exclude single terms

In [18]:
# Load dictionary (with freq>=2)
file_path = 'C:\\Users\\Denis_Davydov2\\OneDrive - EPAM\\Prophet_AI_docs\\Datasets\\AI_skills\\Topics\\'
file_name = 'dic_topics_2025-11-08.csv'

dic_topics = pd.read_csv(file_path+file_name)
print("File read: ", file_name)
dic_topics

File read:  dic_topics_2025-11-08.csv


,topic,count
0,prompt engineering,3255
1,natural language processing (nlp),2996
2,computer vision,2611
3,large language model (llm),2569
4,llm api,2445
...,...,...
7453,nodes,2
7454,noise injection,2
7455,non-autoregressive modeling,2
7456,nope,2


In [43]:
topics = topics_all.merge(dic_topics, how='outer', left_on='term', right_on='topic')
print(f"Rows after merge: {len(topics)}")
# Drop rows where 'topic' is NaN
topics = topics.dropna()
print(f"Rows after removing NaN topics: {len(topics)}")

# Drop duplicates
# Remove leading/trailing spaces from 'term' column and drop duplicates
topics['term'] = topics['term'].str.strip()
topics = topics.drop_duplicates(subset=['topic']).reset_index(drop=True)
print(f"Rows after removing duplicates: {len(topics)}")
topics = topics.drop(columns=['topic'])
topics.dropna(inplace=True)

topics

Rows after merge: 19041
Rows after removing NaN topics: 7431
Rows after removing duplicates: 7431


,id,term,definition,count
0,6.0,1-bit inference,"In the context of AI, 1-bit inference refers t...",2.0
1,7.0,1-bit llm,A 1-bit large language model (LLM) is a highly...,5.0
2,9.0,1-bit quantization,"In the context of AI, 1-bit quantization is an...",6.0
3,14.0,2-bit quantization,"In the context of AI, 2-bit quantization is a ...",4.0
4,17.0,2d animation,"In the context of AI, ""2D animation"" refers to...",3.0
...,...,...,...,...
7426,19003.0,zero-shot voice cloning,Zero-shot voice cloning is the task of synthes...,3.0
7427,19007.0,zero-trust security,Zero-trust security for AI/ML is an architectu...,4.0
7428,19010.0,zero/zero++,"""Zero"" (zero-shot) refers to applying or evalu...",2.0
7429,19012.0,zerogpu,"Zerogpu (often stylized ""zerogpu"") refers to C...",2.0


## Clean list of terms

In [22]:
# Filter topics - include patterns but exclude false positives
# First, apply positive filters
filtered_topics = topics[topics['definition'].str.contains('is not a|has no|does not have a canonical|There is no established', case=False, na=False)]

# Then exclude rows containing these false-positive fragments
exclusion_patterns = [
    'and that it has not',
    'when it is not',
    'that is not',
    'is not affected',
    'to visit states it has not frequently',
    'is not available',
    'and has not been',
    'has not yet',
    'is not a single',
    'has not been'
]

# Build exclusion regex pattern
exclusion_regex = '|'.join([re.escape(pattern) for pattern in exclusion_patterns])

# Apply exclusion filter
filtered_topics = filtered_topics[~filtered_topics['definition'].str.contains(exclusion_regex, case=False, na=False)]

print(f"Found {len(filtered_topics)} topics after applying filters and exclusions")
filtered_topics

Found 65 topics after applying filters and exclusions


,id,term,definition,count
235,618.0,agones,"In the context of AI, Agones is not an artific...",2.0
580,1522.0,ai-gradio,"Based on my research, the term ""ai-gradio"" is ...",2.0
848,2229.0,athena,"In the context of AI, ""Athena"" is not a univer...",3.0
1000,2626.0,avc-lm,Based on my assessment of current AI terminolo...,3.0
1135,2937.0,bfloat11,There appears to be a misunderstanding in your...,2.0
...,...,...,...,...
7362,18850.0,windsurf,"""Windsurf"" is not a standardized term in AI/ML...",3.0
7389,18924.0,xess,"""xess"" is not an established or widely recogni...",2.0
7399,18940.0,xtc sampler,"""XTC sampler"" is not a standard or widely reco...",2.0
7400,18941.0,xtc sampling,"""xtc sampling"" is not a recognized or standard...",3.0


### Non recognized topics

In [21]:
# Save not recognosed topics
file_path = r'C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Topics'
file_name = f'not_recognised.csv'

filtered_topics.to_csv(file_path + '\\' + file_name, index=False, encoding='utf-8-sig')
print(f"File {file_name} \nuploaded to:", file_path)
print("Total rows", len(filtered_topics))

File not_recognised.csv 
uploaded to: C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills\Topics
Total rows 236


In [36]:
# Display all filtered_topics rows in full (no truncation)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
print(f"Total filtered topics: {len(filtered_topics)}\n")
display(filtered_topics)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

Total filtered topics: 109



,id,term,definition
2617,2626,avc-lm,"""avc-lm"" is not a standard, widely recognized acronym or term within the general field of Artificial Intelligence and Machine Learning."
2928,2937,bfloat11,"Bfloat11 is not a widely recognized or standardized floating-point data type within AI/Machine Learning, unlike `bfloat16` or `float32`. While custom 11-bit numerical formats might be explored in specific research or hardware, it lacks a common definition or widespread adoption in the field."
3048,3057,birl,"""Birl"" is not a recognized or standard technical term within the field of Artificial Intelligence or Machine Learning. It does not refer to a specific algorithm, concept, or technique in AI/ML literature or practice."
3515,3524,clairs,"""Clairs"" is not a standard or widely recognized technical term within the field of Artificial Intelligence or Machine Learning. It does not refer to a specific algorithm, concept, or methodology in the industry-standard lexicon."
4487,4496,cork dampening,"""Cork dampening"" is not a standard or recognized term within the fields of Artificial Intelligence or Machine Learning. It does not refer to any specific algorithm, technique, or concept in these domains."
5642,5651,dior,"In the context of AI and Machine Learning, ""dior"" does not have a recognized or defined technical meaning. It is not an acronym, algorithm, or concept within the field."
5664,5673,discopop,"""Discopop"" is not an established or recognized term within the fields of Artificial Intelligence or Machine Learning. It appears to be a misapplication or an invented term unrelated to standard AI/ML concepts or methodologies."
5950,5959,dye-sub pbt,"""Dye-sub PBT"" is a term used to describe a manufacturing process and material for keycaps, commonly found on mechanical keyboards. This term has no relevance or application within the fields of Artificial Intelligence or Machine Learning."
6356,6365,eni),"The term ""eni)"" is not a recognized or standard acronym, abbreviation, or concept within the fields of Artificial Intelligence or Machine Learning. There is no established definition for ""eni)"" in this context."
6711,6720,fairgame,"The term ""fairgame"" is not a standardized technical concept or widely recognized idiom within the field of Artificial Intelligence or Machine Learning. When encountered, it typically refers to a general English expression implying that a particular dataset, model, or AI application is considered an acceptable target for scrutiny, competition, or exploitation, rather than denoting a specific AI methodology or ethical principle."


In [24]:
file_path = 'C:\\Users\\Denis_Davydov2\\OneDrive - EPAM\\Prophet_AI_docs\\Datasets\\AI_skills\\Topics\\'
file_name = 'topics_definitions_corrected.csv'


correction_topics = pd.read_csv(file_path+file_name)
print("File read: ", file_name)
correction_topics

File read:  topics_definitions_corrected.csv


,id,term,definition
0,244,active recall,"In AI/Machine Learning, ""active recall"" refers..."
1,618,agones,"The term ""agones"" does not have a widely recog..."
2,1522,ai-gradio,Gradio is an open-source Python library that e...
3,1710,almond,Almond is an open-source virtual assistant pla...
4,1959,any,"In AI and Machine Learning, 'any' most commonl..."
...,...,...,...
231,18965,zenmcp,"The term ""zenmcp"" is not recognized as a stand..."
232,18966,zephyr,"Zephyr is a series of small, supervised fine-t..."
233,18977,zero++,Zero++ is an AI approach that extends zero-sho...
234,19013,zig,"The term ""zig"" does not have a widely recogniz..."


In [25]:
# Replace definitions in topics using correction_topics based on (id, term)
import pandas as pd

required_cols = {'id','term','definition'}
if not required_cols.issubset(topics.columns):
    raise ValueError(f"'topics' is missing required columns: {required_cols - set(topics.columns)}")
if not required_cols.issubset(correction_topics.columns):
    raise ValueError(f"'correction_topics' is missing required columns: {required_cols - set(correction_topics.columns)}")

# Prepare correction index (drop duplicate (id,term) keeping the last non-empty definition)
ct = correction_topics.copy()
ct = ct.dropna(subset=['id','term'])
ct['term'] = ct['term'].astype(str).str.strip()
ct['definition'] = ct['definition'].astype(str)
# Keep last occurrence for each (id,term)
ct = ct.sort_values(['id','term']).drop_duplicates(subset=['id','term'], keep='last')
ct_idx = ct.set_index(['id','term'])['definition']

# Set index on topics for alignment
orig = topics.copy()
topics_idx = topics.set_index(['id','term'])

# Track before/after for stats
before_missing = (topics_idx['definition'].isna() | (topics_idx['definition'].astype(str).str.strip()=='' )).sum()
updates = 0

# Apply replacements where correction definition is non-empty and different
for key, new_def in ct_idx.items():
    if key in topics_idx.index:
        current_def = str(topics_idx.at[key,'definition'])
        if new_def and new_def.strip() and new_def.strip() != current_def.strip():
            topics_idx.at[key,'definition'] = new_def.strip()
            updates += 1

# Restore original index
topics = topics_idx.reset_index()

after_missing = (topics['definition'].isna() | (topics['definition'].astype(str).str.strip()=='' )).sum()

print(f"Definitions updated: {updates}")
print(f"Missing/empty before: {before_missing} | after: {after_missing}")
print(f"Total rows: {len(topics)}")

# Preview first 15 updated rows where definitions changed
changed = topics.merge(orig[['id','term','definition']].rename(columns={'definition':'old_definition'}), on=['id','term'])
changed = changed[changed['definition'].astype(str).str.strip() != changed['old_definition'].astype(str).str.strip()]
print(f"Rows with changed definition: {len(changed)}")
changed.head(15)

Definitions updated: 65
Missing/empty before: 0 | after: 0
Total rows: 7431
Rows with changed definition: 65


,id,term,definition,count,old_definition
235,618.0,agones,"The term ""agones"" does not have a widely recog...",2.0,"In the context of AI, Agones is not an artific..."
580,1522.0,ai-gradio,Gradio is an open-source Python library that e...,2.0,"Based on my research, the term ""ai-gradio"" is ..."
848,2229.0,athena,"In AI/Machine Learning contexts, ""Athena"" most...",3.0,"In the context of AI, ""Athena"" is not a univer..."
1000,2626.0,avc-lm,"""avc-lm"" is not a standard, widely recognized ...",3.0,Based on my assessment of current AI terminolo...
1135,2937.0,bfloat11,Bfloat11 is not a widely recognized or standar...,2.0,There appears to be a misunderstanding in your...
2193,5585.0,differential transformer,"A differential transformer, in the context of ...",3.0,"A ""differential transformer"" is not a recogniz..."
2607,6720.0,fairgame,"The term ""fairgame"" is not a standardized tech...",2.0,"In the context of AI and Machine Learning, ""fa..."
2716,7007.0,flowspec,"In AI/Machine Learning, ""flowspec"" refers to t...",2.0,"The term ""flowspec"" does not have a recognized..."
2719,7012.0,flux.1 kontext,Flux.1 kontext might refer to a specific versi...,2.0,"""Flux.1 kontext"" is not a recognized or indust..."
2783,7191.0,fvd,Fréchet Video Distance (FVD) is a metric used ...,2.0,"""FVD"" is not a standard or widely recognized a..."


In [26]:
topics

,id,term,definition,count
0,6.0,1-bit inference,"In the context of AI, 1-bit inference refers t...",2.0
1,7.0,1-bit llm,A 1-bit large language model (LLM) is a highly...,5.0
2,9.0,1-bit quantization,"In the context of AI, 1-bit quantization is an...",6.0
3,14.0,2-bit quantization,"In the context of AI, 2-bit quantization is a ...",4.0
4,17.0,2d animation,"In the context of AI, ""2D animation"" refers to...",3.0
...,...,...,...,...
7426,19003.0,zero-shot voice cloning,Zero-shot voice cloning is the task of synthes...,3.0
7427,19007.0,zero-trust security,Zero-trust security for AI/ML is an architectu...,4.0
7428,19010.0,zero/zero++,"""Zero"" (zero-shot) refers to applying or evalu...",2.0
7429,19012.0,zerogpu,"Zerogpu (often stylized ""zerogpu"") refers to C...",2.0


In [36]:
# Delete rows from topics that exist in filtered_topics based on (id, term)
topics_before = len(topics)

# Create a set of (id, term) tuples from filtered_topics for efficient lookup
filtered_keys = set(zip(filtered_topics['id'], filtered_topics['term']))

# Filter out rows from topics where (id, term) exists in filtered_topics
topics = topics[~topics.set_index(['id', 'term']).index.isin(filtered_keys)]

topics_after = len(topics)
removed = topics_before - topics_after

print(f"Rows before: {topics_before}")
print(f"Rows after: {topics_after}")
print(f"Rows removed: {removed}")

Rows before: 7431
Rows after: 7366
Rows removed: 65


### Correct defenitions

In [37]:
topics

,id,term,definition,count
0,6.0,1-bit inference,"In the context of AI, 1-bit inference refers t...",2.0
1,7.0,1-bit llm,A 1-bit large language model (LLM) is a highly...,5.0
2,9.0,1-bit quantization,"In the context of AI, 1-bit quantization is an...",6.0
3,14.0,2-bit quantization,"In the context of AI, 2-bit quantization is a ...",4.0
4,17.0,2d animation,"In the context of AI, ""2D animation"" refers to...",3.0
...,...,...,...,...
7426,19003.0,zero-shot voice cloning,Zero-shot voice cloning is the task of synthes...,3.0
7427,19007.0,zero-trust security,Zero-trust security for AI/ML is an architectu...,4.0
7428,19010.0,zero/zero++,"""Zero"" (zero-shot) refers to applying or evalu...",2.0
7429,19012.0,zerogpu,"Zerogpu (often stylized ""zerogpu"") refers to C...",2.0


In [38]:
# Remove unwanted  prefixes from defenition
unwanted_prefixes = [
    'Answer: ',
    'In the context of artificial intelligence, ',
    'In the context of AI/Machine Learning, ',
    'In AI/ML, ',
    'In the context of AI, ',
    'In the AI/ML context, ',
    'In the context of AI and machine learning, ',
    'In machine learning, ',
    'In artificial intelligence, ',
    'In AI, ',
    'In the context of machine learning, ',
    'In the context of AI/ML, ',
    'In the context of AI and ML, ',
    'In the context of artificial intelligence and machine learning, ',
    'In the context of AI/ML, ',
    'In AI and machine learning, '
]

# Strip any leading/trailing whitespace
topics['definition'] = topics['definition'].str.strip()

# Apply deletion
pattern = r'(?i)^(?:' + '|'.join(re.escape(p) for p in unwanted_prefixes) + ')'
topics1 = topics.copy()
topics1['definition'] = (
    topics1['definition']
        .astype(str)
        .str.strip()
        .str.replace(pattern, '', regex=True)
        .str.strip()
)

# Strip any leading/trailing whitespace created by removal
topics1['definition'] = topics1['definition'].str.strip()


dups_before = topics1['definition'].duplicated().sum()
print(f"Duplicate definitions before dedup: {dups_before}")
before_rows = len(topics1)
topics1 = topics1.drop_duplicates(subset=['definition']).reset_index(drop=True)
after_rows = len(topics1)
print(f"Rows before: {before_rows} | after: {after_rows} | removed: {before_rows - after_rows}")



topics1

Duplicate definitions before dedup: 0
Rows before: 7366 | after: 7366 | removed: 0


C:\Users\Denis_Davydov2\AppData\Local\Temp\ipykernel_22056\3193863041.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topics['definition'] = topics['definition'].str.strip()


,id,term,definition,count
0,6.0,1-bit inference,1-bit inference refers to the process of runni...,2.0
1,7.0,1-bit llm,A 1-bit large language model (LLM) is a highly...,5.0
2,9.0,1-bit quantization,1-bit quantization is an extreme model compres...,6.0
3,14.0,2-bit quantization,2-bit quantization is a process that reduces t...,4.0
4,17.0,2d animation,"""2D animation"" refers to the use of artificial...",3.0
...,...,...,...,...
7361,19003.0,zero-shot voice cloning,Zero-shot voice cloning is the task of synthes...,3.0
7362,19007.0,zero-trust security,Zero-trust security for AI/ML is an architectu...,4.0
7363,19010.0,zero/zero++,"""Zero"" (zero-shot) refers to applying or evalu...",2.0
7364,19012.0,zerogpu,"Zerogpu (often stylized ""zerogpu"") refers to C...",2.0


In [33]:
topics1[topics1['id'] == 649]

,id,term,definition,count
245,649.0,ai agent development,"agent development is the process of designing,...",95.0


# Embedding (OpenAI text-embedding)

In [39]:
topic_items = topics1.copy()
#topic_items = topic_items.head(25)
#topic_items = topic_items.iloc[9474:9475]
topic_items

,id,term,definition,count
0,6.0,1-bit inference,1-bit inference refers to the process of runni...,2.0
1,7.0,1-bit llm,A 1-bit large language model (LLM) is a highly...,5.0
2,9.0,1-bit quantization,1-bit quantization is an extreme model compres...,6.0
3,14.0,2-bit quantization,2-bit quantization is a process that reduces t...,4.0
4,17.0,2d animation,"""2D animation"" refers to the use of artificial...",3.0
...,...,...,...,...
7361,19003.0,zero-shot voice cloning,Zero-shot voice cloning is the task of synthes...,3.0
7362,19007.0,zero-trust security,Zero-trust security for AI/ML is an architectu...,4.0
7363,19010.0,zero/zero++,"""Zero"" (zero-shot) refers to applying or evalu...",2.0
7364,19012.0,zerogpu,"Zerogpu (often stylized ""zerogpu"") refers to C...",2.0


In [40]:
# === Load Environment Variables ===
load_dotenv()  # Loads variables from .env file into environment

# Retrieve database connection parameters from environment variables
DIAL_API_KEY = os.getenv('DIAL_API_KEY')

#print(DIAL_API_KEY)

In [42]:
# Batch embedding generation for (term + definition) pairs
import json
import math
import time
from datetime import date

# Configuration
EMBED_MODEL = "text-embedding-3-small-1"
AZURE_OPENAI_ENDPOINT = "https://ai-proxy.lab.epam.com"
API_VERSION = "2025-04-01-preview"
BATCH_SIZE = 500
SLEEP_BETWEEN_CALLS = 1.0
OUTPUT_EMB_FILE = f"topic_embeddings_{date.today()}.csv"

# Preconditions
required_cols = {"id","term","definition"}
if not required_cols.issubset(topic_items.columns):
    raise ValueError(f"'topics' DataFrame missing columns: {required_cols - set(topic_items.columns)}")

DIAL_API_KEY = os.getenv("DIAL_API_KEY")
if not DIAL_API_KEY:
    raise EnvironmentError("DIAL_API_KEY not found in environment variables.")

# Initialize Azure OpenAI Embedding client
from openai import AzureOpenAI
emb_client = AzureOpenAI(
    api_key=DIAL_API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

# Prepare texts: first line term, second line definition
work_df = topic_items.copy()
work_df["term"] = work_df["term"].astype(str).str.strip()
work_df["definition"] = work_df["definition"].fillna("").astype(str).str.strip()

texts = [f"{t}\n{d}" for t,d in zip(work_df["term"], work_df["definition"])]
print(f"Total texts to embed: {len(texts)}")

embeddings_records = []
num_batches = math.ceil(len(texts)/BATCH_SIZE)
print(f"Processing in {num_batches} batches (size={BATCH_SIZE}) using model '{EMBED_MODEL}'\n")
start_ts = time.time()

for b in range(num_batches):
    start = b*BATCH_SIZE
    end = min((b+1)*BATCH_SIZE, len(texts))
    batch_texts = texts[start:end]
    batch_ids = work_df["id"].iloc[start:end].tolist()
    batch_terms = work_df["term"].iloc[start:end].tolist()
    batch_defs = work_df["definition"].iloc[start:end].tolist()
    batch_counts = work_df["count"].iloc[start:end].tolist() if "count" in work_df.columns else [None]*len(batch_texts)

    try:
        resp = emb_client.embeddings.create(model=EMBED_MODEL, input=batch_texts)
        # resp.data is a list with .embedding
        if not resp or not hasattr(resp, "data"):
            raise RuntimeError("Unexpected embedding response format")
        vectors = [item.embedding for item in resp.data]
    except Exception as e:
        print(f"\nError embedding batch {b+1}/{num_batches}: {e}")
        vectors = [None]*len(batch_texts)

    for i,(row_id, term, definition, count, vec) in enumerate(zip(batch_ids, batch_terms, batch_defs, batch_counts, vectors)):
        embeddings_records.append({
            "id": row_id,
            "term": term,
            "definition": definition,
            "count": count,
            "embedding": json.dumps(vec) if vec is not None else ""
        })

    # Progress output
    if vectors and vectors[0] is not None:
        dim = len(vectors[0])
    else:
        dim = "?"
    print(f"Batch {b+1}/{num_batches} done: rows {start}-{end-1} | dim={dim}")
    time.sleep(SLEEP_BETWEEN_CALLS)

elapsed = time.time() - start_ts
print(f"\nEmbedding complete in {elapsed:.2f}s")

# Build embeddings DataFrame
emb_df = pd.DataFrame(embeddings_records)
non_empty = (emb_df["embedding"].str.len() > 0).sum()
print(f"Rows embedded (non-empty vectors): {non_empty}/{len(emb_df)}")

# Persist to CSV (embedding stored as JSON array string)
emb_df.to_csv(OUTPUT_EMB_FILE, index=False, encoding="utf-8-sig")
print(f"Saved embeddings to: {OUTPUT_EMB_FILE}")

# Preview
emb_df

Total texts to embed: 7366
Processing in 15 batches (size=500) using model 'text-embedding-3-small-1'

Batch 1/15 done: rows 0-499 | dim=1536
Batch 1/15 done: rows 0-499 | dim=1536
Batch 2/15 done: rows 500-999 | dim=1536
Batch 2/15 done: rows 500-999 | dim=1536
Batch 3/15 done: rows 1000-1499 | dim=1536
Batch 3/15 done: rows 1000-1499 | dim=1536
Batch 4/15 done: rows 1500-1999 | dim=1536
Batch 4/15 done: rows 1500-1999 | dim=1536
Batch 5/15 done: rows 2000-2499 | dim=1536
Batch 5/15 done: rows 2000-2499 | dim=1536
Batch 6/15 done: rows 2500-2999 | dim=1536
Batch 6/15 done: rows 2500-2999 | dim=1536
Batch 7/15 done: rows 3000-3499 | dim=1536
Batch 7/15 done: rows 3000-3499 | dim=1536
Batch 8/15 done: rows 3500-3999 | dim=1536
Batch 8/15 done: rows 3500-3999 | dim=1536
Batch 9/15 done: rows 4000-4499 | dim=1536
Batch 9/15 done: rows 4000-4499 | dim=1536
Batch 10/15 done: rows 4500-4999 | dim=1536
Batch 10/15 done: rows 4500-4999 | dim=1536
Batch 11/15 done: rows 5000-5499 | dim=1536
Bat

,id,term,definition,count,embedding
0,6.0,1-bit inference,1-bit inference refers to the process of runni...,2.0,"[0.013369596563279629, -0.013582167215645313, ..."
1,7.0,1-bit llm,A 1-bit large language model (LLM) is a highly...,5.0,"[0.017574984580278397, -0.004011554177850485, ..."
2,9.0,1-bit quantization,1-bit quantization is an extreme model compres...,6.0,"[-0.01522609032690525, 0.0012089558877050877, ..."
3,14.0,2-bit quantization,2-bit quantization is a process that reduces t...,4.0,"[-0.006645898800343275, 0.006167215760797262, ..."
4,17.0,2d animation,"""2D animation"" refers to the use of artificial...",3.0,"[0.008917991071939468, -0.002153154695406556, ..."
...,...,...,...,...,...
7361,19003.0,zero-shot voice cloning,Zero-shot voice cloning is the task of synthes...,3.0,"[-0.007965236902236938, -0.027662580832839012,..."
7362,19007.0,zero-trust security,Zero-trust security for AI/ML is an architectu...,4.0,"[-0.01758248545229435, 0.059987302869558334, 0..."
7363,19010.0,zero/zero++,"""Zero"" (zero-shot) refers to applying or evalu...",2.0,"[-0.0007028839900158346, 0.011448170058429241,..."
7364,19012.0,zerogpu,"Zerogpu (often stylized ""zerogpu"") refers to C...",2.0,"[0.004047493916004896, 0.05906970053911209, 0...."


# Target vector: 'AI'

In [1]:
target_ai = '''Development and implementation of artificial intelligence
The development and implementation of AI involve defining goals, collecting and preparing data, building and testing models, and deploying and monitoring the final system. A successful implementation starts with clear objectives, progresses through a structured lifecycle of data handling and model refinement, and includes critical steps like risk assessment, ethical considerations, and continuous monitoring to ensure the AI system functions correctly and safely. 
Development lifecycle
Planning and goal definition: Identify specific problems or opportunities, set measurable objectives, and create a project roadmap with timelines and risk assessments.
Data collection and preparation: Gather necessary data and then clean, process, and annotate it as needed for training.
Model development: Select an appropriate algorithm, build and compare multiple models, and perform hyperparameter tuning to optimize performance.
Model evaluation and testing: Validate the model's accuracy using training, validation, and testing data sets, employing techniques like cross-validation and benchmarking. Also, conduct bias and fairness testing. 
Implementation and deployment
Deployment: Integrate the AI model into existing systems using APIs, microservices, or containerization. Decide on an infrastructure (cloud, on-premises, or hybrid) and ensure the solution is scalable and secure.
Post-deployment monitoring: Continuously monitor the AI system's performance to maintain accuracy and retrain or update it as needed. 
'''

In [2]:
print(target_ai)

Development and implementation of artificial intelligence
The development and implementation of AI involve defining goals, collecting and preparing data, building and testing models, and deploying and monitoring the final system. A successful implementation starts with clear objectives, progresses through a structured lifecycle of data handling and model refinement, and includes critical steps like risk assessment, ethical considerations, and continuous monitoring to ensure the AI system functions correctly and safely. 
Development lifecycle
Planning and goal definition: Identify specific problems or opportunities, set measurable objectives, and create a project roadmap with timelines and risk assessments.
Data collection and preparation: Gather necessary data and then clean, process, and annotate it as needed for training.
Model development: Select an appropriate algorithm, build and compare multiple models, and perform hyperparameter tuning to optimize performance.
Model evaluation a

In [7]:
import json
import math
import time
from datetime import date

# Configuration
EMBED_MODEL = "text-embedding-3-small-1"
AZURE_OPENAI_ENDPOINT = "https://ai-proxy.lab.epam.com"
API_VERSION = "2025-04-01-preview"
BATCH_SIZE = 500
SLEEP_BETWEEN_CALLS = 1.0
OUTPUT_EMB_FILE = f"topic_embeddings_{date.today()}.csv"


DIAL_API_KEY = os.getenv("DIAL_API_KEY")


# Initialize Azure OpenAI Embedding client
from openai import AzureOpenAI
emb_client = AzureOpenAI(
    api_key=DIAL_API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)


start_ts = time.time()

resp = emb_client.embeddings.create(model=EMBED_MODEL, input=target_ai)
ai_embedding = [item.embedding for item in resp.data]

elapsed = time.time() - start_ts
print(f"\nEmbedding complete in {elapsed:.2f}s")
print(ai_embedding)


Embedding complete in 1.00s
[[-0.00029644169262610376, -0.00554628437384963, 0.05506531894207001, -0.006948511581867933, 0.028164342045783997, -0.014321775175631046, 0.011163360439240932, 0.02352474071085453, 0.006278709974139929, 0.051580172032117844, 0.05480393394827843, -0.05166730284690857, -0.04134255275130272, -0.07297026365995407, -0.01988711953163147, -0.027336619794368744, -0.029493054375052452, 0.019407911226153374, 0.024722760543227196, 0.017447516322135925, 0.038118794560432434, 0.030821766704320908, 0.019440583884716034, 0.04609106481075287, -0.0006020727450959384, -0.027184145525097847, 0.011871281079947948, 0.040384139865636826, -0.029340580105781555, 0.00951880682259798, 0.02988513372838497, -0.008789104409515858, -0.037922754883766174, 0.011740588583052158, 0.021967316046357155, 0.01022128202021122, -0.011533658020198345, 0.0029433153104037046, -0.0063331653364002705, 0.027859391644597054, 0.021793058142066002, -0.008217322640120983, -0.02598612569272518, 0.0322376079

In [12]:
ai_embedding = ai_embedding[0]
ai_embedding

[-0.00029644169262610376,
 -0.00554628437384963,
 0.05506531894207001,
 -0.006948511581867933,
 0.028164342045783997,
 -0.014321775175631046,
 0.011163360439240932,
 0.02352474071085453,
 0.006278709974139929,
 0.051580172032117844,
 0.05480393394827843,
 -0.05166730284690857,
 -0.04134255275130272,
 -0.07297026365995407,
 -0.01988711953163147,
 -0.027336619794368744,
 -0.029493054375052452,
 0.019407911226153374,
 0.024722760543227196,
 0.017447516322135925,
 0.038118794560432434,
 0.030821766704320908,
 0.019440583884716034,
 0.04609106481075287,
 -0.0006020727450959384,
 -0.027184145525097847,
 0.011871281079947948,
 0.040384139865636826,
 -0.029340580105781555,
 0.00951880682259798,
 0.02988513372838497,
 -0.008789104409515858,
 -0.037922754883766174,
 0.011740588583052158,
 0.021967316046357155,
 0.01022128202021122,
 -0.011533658020198345,
 0.0029433153104037046,
 -0.0063331653364002705,
 0.027859391644597054,
 0.021793058142066002,
 -0.008217322640120983,
 -0.02598612569272518,


In [14]:
import json

file_path = 'C:\\Users\\Denis_Davydov2\\OneDrive - EPAM\\Prophet_AI_docs\\Datasets\\AI_skills\\Topics\\'
file_name = f'target_AI_vector.txt'

# Save as text file with JSON array
with open(file_path + file_name, 'w', encoding='utf-8') as f:
    json.dump(ai_embedding, f)

print(f"File {file_name} saved: ", date.today())
print('File uploaded successfully')
print(f"Vector dimension: {len(ai_embedding)}")

File target_AI_vector.txt saved:  2025-11-09
File uploaded successfully
Vector dimension: 1536
